In [1]:
%load_ext autoreload
from numpy.fft import fft2, ifft2, fftshift
import matplotlib
matplotlib.use('Qt4Agg')
matplotlib.interactive(True)
# %matplotlib inline
import mayavi.mlab as mlab
import matplotlib.pyplot as plt
# import matplotlib.cm as cm
import numpy as np
from scipy import signal

import pyfpm.fpmmath as fpm
import pyfpm.data as dt
import pyfpm.local as local


('/home/lec/pyfpm', '/home/lec/pyfpm/etc/config.yaml')


In [3]:
import pyfpm.coordtrans as ct

def set_setup(cfg):
    """ Sets up the scene.
    """
    tan_theta = 0.1/(np.sqrt(1-0.1**2)) # sin/cos
    na_radius = tan_theta*cfg.sample_height
    x0, y0 ,z0 = [0, 0, 0]
    xs, ys, zs = [0, 0, -cfg.sample_height]
    mlab.figure(size=(600,600), bgcolor=(1, 1, 1))

    mlab.points3d(x0, y0, z0, 1, colormap="hot", mode='2dsquare', scale_factor=40, vmin=1, vmax=3)
    mlab.points3d(x0, y0, z0, 1, colormap="hot", mode='axes', scale_factor=15, vmin=1, vmax=3)
    mlab.points3d(xs, ys, zs, 2.1, colormap="hot", mode='axes', scale_factor=8, vmin=1, vmax=3)
    # Na circle projected in source plane
    t = np.linspace(0, 2*np.pi, 100)
    mlab.plot3d(na_radius*np.sin(t), na_radius*np.cos(t), -cfg.sample_height*np.ones_like(t), tube_radius=1)
    camview = mlab.view()
    fig = mlab.gcf()
    return fig, camview

def plot_source(position, direction, camview, color='None', fig='None'):
    x, y ,z = position
    u, v, w = direction
    value = 1.2
    mlab.points3d(x, y, -z, value, color=color, mode='sphere', scale_factor=2, vmin = 2, vmax = 3, figure=fig)
    mlab.quiver3d(x, y, -z, u, v , w, line_width=2.2, color=(.1, .1, 1), scale_factor=5, figure=fig)
    mlab.view(*camview)
    return

def plot_sample_plane(position, direction):
    xspot, yspot, zspot = ct.calculate_spot_center(position, direction)
    mlab.points3d(xspot, yspot, zspot, 2, color=color, mode='2dcircle', scale_factor=.5, vmin=0, vmax=1, figure=fig)
    spot_image = ct.spot_image(position, direction)
#     mlab.imshow(spot_image, extent=[-20, 20, -20, 20, 0, 0], figure=fig)
    return

# Simulation parameters
cfg = dt.load_config()
simclient = local.SimClient(cfg=cfg)
fig, camview = set_setup(cfg)
iterator = fpm.set_iterator(cfg)
light_dir = [0, 0, 1]
source_center = [0, 0]
source_tilt = [0, 0]
platform_tilt = [0, 10]
for index, theta, phi, acqpars in iterator:
    plat_coordinates = ct.input_angles_to_platform(theta, phi, cfg.sample_height)
    position, direction = ct.platform_to_cartesian(plat_coordinates, light_dir, 
                          source_center, source_tilt, platform_tilt, cfg.sample_height)
    color = (1.*phi/cfg.phi[1], 1-1.*phi/cfg.phi[1], .1)
    direction = ct.light_dir_from_angles(theta, phi, source_tilt)
    plot_source(position, direction, camview, color, fig)
    plot_sample_plane(position, direction)
    
# arr = mlab.screenshot()
# mlab.close()
# fig, ax = plt.subplots(1,1, figsize=[10, 10])
# ax.imshow(arr)
# ax.axis('off')
# plt.show()

/usr/local/lib/python2.7/dist-packages/mayavi/tools/camera.py:288: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if focalpoint is not None and not focalpoint == 'auto':
